# Лабораторная работа №3
## Выполнил студент группы БФИ2001 Аржанова Анастасия

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [1]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [9]:
def prefix_function(text):#составляет список из 0 и 1, 0 - значит буквы не один, 1 - буквы ранвы
    n = len(text)
    pi = [0] * n
    for i in range(1, n):
        j = pi[i-1]
        while j > 0 and text[j] != text[i]:
            j = pi[j-1]
        if text[i] == text[j]:
            j += 1
        pi[i] = j
    return pi


def kmp_search(text, sub_text, start_index=0):
    ch = 0
    pi = prefix_function(sub_text)
    for i in range(start_index, len(text)):# проходимся по тексту
        while ch > 0 and text[i] != sub_text[ch]:
            ch = pi[ch-1]
        if text[i] == sub_text[ch]:# сравниваются 2 буквы слов
            ch += 1
        if ch >= len(sub_text):# если счетчик сравнялся или получился больше с длиной введенного слова
            return "Результат: Совпадение найдено."
    return "Результат: Совпадение не найдено"# если после прохождения слова счетчик не сравнялся с доп словом


text = input("введите строку: ")
sub_text = input("введите слова для находения: ")

print(kmp_search(text, sub_text, 0))

ананас
на
0 н а
Результат: Совпадение найдено.


#### Упрощенный алгоритм Бойера-Мура

In [13]:
# Этап 1: формирование таблицы смещений
def bmPredCompil(sub_text):
    S = set()  # создание множества
    M = len(sub_text) # длина слова
    dictionary = {} # словарь смещений
    for i in range(M-2, -1, -1): # итерации с предпоследнего символа
        if sub_text[i] not in S:        # если символ еще не добавлен в таблицу
            dictionary[sub_text[i]] = M-i-1# записываем под ключем буквой, значение равное м-i-1
            S.add(sub_text[i])# записываем саму букву
    if sub_text[M-1] not in S:     # отдельно формируем последний символ
        dictionary[sub_text[M-1]] = M
    dictionary['*'] = M              # смещения для прочих символов
    return dictionary

# Этап 2: поиск образа в строке
def boyerMurSearch(text, sub_text):
    dictionary = bmPredCompil(sub_text) #формирование таблицы смещений
    N = len(text)  # длина подного текса
    M = len(sub_text) # длина поискогого слова
    if N >= M:
        i = M-1       # счетчик проверяемого символа в строке
        while(i < N):
            k = 0
            j = 0
            f = False
            for j in range(M-1, -1, -1):
                if text[i-k] != sub_text[j]:
                    if j == M-1:
                        off = dictionary[text[i]] if dictionary.get(text[i], False) else dictionary['*']  
                        # смещение, если не равен последний символ образа
                    else:
                        off = dictionary[sub_text[j]]   
                        # смещение, если не равен не последний символ образа

                    i += off    # смещение счетчика строки
                    f = True  # если несовпадение символа, то flBreak = True
                    break
                k += 1          # смещение для сравниваемого символа в строке
            if not f:          # если дошли до начала образа, значит, все его символы совпали
                return "Результат: Совпадение найдено"
                break
        else:
            return "Результат: Совпадение не найдено"
    else:
        return "Результат: Совпадение не найдено"
    
    
text = input("Введите строку: ")
sub_text = input("Введите подстроку: ")

print(boyerMurSearch(text, sub_text))

Введите строку: ананас
Введите подстроку: на
Результат: Совпадение найдено


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [23]:
from heapq import heappop, heappush
from math import sqrt
import argparse

def a_star(start_chain, goal_node):
    node_hash = {}
    chains_queue = []
    heappush(chains_queue, start_chain)
    while chains_queue:
        cur_chain = heappop(chains_queue)
        cur_node = cur_chain.last_node()
        if cur_node == goal_node:
            return cur_chain
        node_hash[cur_node] = cur_chain.g()
        for chain in cur_chain.get_neighbours():
            if chain.last_node() in node_hash:
                if chain.g() >= node_hash[chain.last_node()]:
                    continue
                node_hash[chain.last_node()] = chain.g()
            heappush(chains_queue, chain)

    raise Exception("Нет решения!")

def manh_dst_matrix(a, b, n):
    return abs(a % n - b % n) + abs(a // n - b // n)

class chain15:
    def __str__(self): # создаем матрицу
        i = 0
        sstr = ""
        while i < self.size ** 2: # пока ай меньше квадрата длины
            sstr += str(self.board_state[i]) + " "  # записываем число из массива
            if i % self.size == 3: # когда ай будет равен 3 белаем перенос на новую страку, для отрисовки матрицы
                sstr += "\n"
            i += 1 # увеличиваем счетчик
        return sstr #возращаем матрицу записанную ввиде текста

    def __init__(self, board_state, history=[]):
        self.board_state = list(board_state) # создаем список массива
        self.size = int(sqrt(len(board_state))) #расчитываем целое число от корня из длины массива
        self.history = history  # == []
        self.quad_size = int(self.size * self.size) # квадрат длины

    def manh_dst(self):
        dst = 0
        for i in range(0, self.quad_size):
            dst += manh_dst_matrix((self.board_state[i] - 1) % self.quad_size, i, self.size)
        return int(dst)
    
    def last_node(self):
        return str(self.board_state)
    
    def last_move(self):
        if self.board_state[-1] == self.quad_size - 1 or self.board_state[-1] == self.quad_size - self.size:
            return 0
        return 2

    def simple_heur(self):
        dst = 0
        for i in range(0, int(self.quad_size)):
            if (self.board_state[i] - 1) != i:
                dst += 1
        return dst

    def f(self):
        return self.manh_dst() + self.last_move()
    
    def g(self):
        return len(self.history)

    def __lt__(self, other):
        return self.f() < other.f()

    def get_neighbours(self):
        neighs = []
        zero_coord = self.board_state.index(0)

        # look at neighbours
        if zero_coord + 1 < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + 1] = new_state[zero_coord + 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - 1 >= 0 and manh_dst_matrix(zero_coord, zero_coord - 1, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - 1] = new_state[zero_coord - 1], new_state[zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord + self.size < self.size ** 2 and manh_dst_matrix(zero_coord, zero_coord + self.size,
                                                                       self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord + self.size] = new_state[zero_coord + self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        if zero_coord - self.size >= 0 and manh_dst_matrix(zero_coord, zero_coord - self.size, self.size) == 1:
            new_state = self.board_state.copy()
            new_state[zero_coord], new_state[zero_coord - self.size] = new_state[zero_coord - self.size], new_state[
                zero_coord]
            neighs.append(chain15(new_state, self.history + [self]))

        return neighs

start_state = [1, 0, 3, 4, 5, 6, 7, 10, 12, 8, 11, 9, 13, 14, 15, 2]
start = chain15(start_state)# составляем матрицу для начала
end = chain15((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0))# какая матрица у нас должна получится в конце

result = a_star(start, end.last_node())# все действия происходят в а стар

for node in result.history:
    print(str(node))
    print("-------------------------")
print(str(result))

1 0 3 4 
5 6 7 10 
12 8 11 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 0 7 10 
12 8 11 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 0 11 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 0 9 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 9 0 
13 14 15 2 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 9 2 
13 14 15 0 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 9 2 
13 14 0 15 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 0 2 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 7 10 
12 11 2 0 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 7 0 
12 11 2 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 0 7 
12 11 2 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
12 11 0 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
12 0 11 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
0 12 11 10 
13 14 9 15 

-------------------------
1 6 3 4 
5 8 2 7 
13 12 11 10 
0 1

### Вывод